# Web Scraping to complete the dataset

In [30]:
import re
import time
import pandas as pd
import seaborn as sns
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

In [2]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [3]:
df_courses = pd.read_csv('../Data/interim/Courses.csv')

In [4]:
df_courses.head()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,publishedTime,category,timeSpent,publishDate,level,paidBool
0,28295,Learn Web Designing & HTML5/CSS3 Essentials in...,https://www.udemy.com/build-beautiful-html5-we...,True,75.0,43285,525,24,All Levels,4 hours,2013-01-03T00:55:31Z,WebDevelopment,4.0,2013-01-03,All Levels,True
1,19603,Learning Dynamic Website Design - PHP MySQL an...,https://www.udemy.com/learning-dynamic-website...,True,50.0,47886,285,125,All Levels,12.5 hours,2012-06-18T16:52:34Z,WebDevelopment,12.5,2012-06-18,All Levels,True
2,889438,ChatBots: Messenger ChatBot with API.AI and No...,https://www.udemy.com/chatbots/,True,50.0,2577,529,64,All Levels,4.5 hours,2016-06-30T16:57:08Z,WebDevelopment,4.5,2016-06-30,All Levels,True
3,197836,Projects in HTML5,https://www.udemy.com/projects-in-html5/,True,60.0,8777,206,75,Intermediate Level,15.5 hours,2014-06-17T05:43:50Z,WebDevelopment,15.5,2014-06-17,Intermediate Level,True
4,505208,Programming Foundations: HTML5 + CSS3 for Entr...,https://www.udemy.com/html-css-more/,True,20.0,23764,490,58,Beginner Level,5.5 hours,2015-10-17T04:52:25Z,WebDevelopment,5.5,2015-10-17,Beginner Level,True


In [20]:
global headers

In [21]:
headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        }

In [183]:
df_courses.url[2]

'https://www.udemy.com/chatbots/'

In [28]:
req = Request(df_courses.url[0], headers=headers) 
webpage = urlopen(req).read()

In [171]:
page_soup = soup(webpage, 'html.parser')

In [180]:
page_soup


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Learning Dynamic Website Design - PHP MySQL and JavaScript | Udemy</title>
<meta content="Learn Dynamic Web Design with PHP / MySQL, AJAX, JavaScript and CSS. Course comes with Certificate of Completion. " name="description"/><meta content="Learning Dynamic Website Design - PHP MySQL and JavaScript" name="title"/><meta content="summary_large_image" name="twitter:card"/><meta content="mult" name="medium"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/><meta content="app-id=562413829, affiliate-data=ct=Safari_SmartBanner&amp;amp;pt=1240482" name="apple-itunes-app"/><meta content="app-id=com.udemy.android" name="google-play-app"/><meta content="313137469260" property="fb:app_id"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="1" property

# Finding relevant features 

Using this `page_soup` we proceed to extract some features before to create the functions that will replicate the same for all the courses in the catalog of the dataset.

### First Review:

In [173]:
review = page_soup.find('div', class_ ='ud-component--clp--featured-review-content')
review

### Description:

In [215]:
description = page_soup.find('div', class_ ='js-simple-collapse-inner')
description

<div class="js-simple-collapse-inner" data-purpose="collapse-description-text">
<div class="description__title">
Description
</div>
<div class="">
<h5>A Verifiable Certificate of Completion is presented to all students who undertake this course.</h5> <p>This Dynamic Web Design with PHP and MySQL training course will give you a working understanding of these important technologies and show you how to use them to create your own impressive websites. When used in combination, there simply isn't a better way to construct sites that involve complexity, large volumes of data, and the flexibility to make adjustments and changes over time. This course goes step by step through the basics of each technology, and then gradually shows you how to build the necessary components and underlying framework of your site. It goes through installation of PHP and MySQL technologies on your local machine as well as hosted servers, explains basic tags, commands and constructs, and helps you build and work wi

In [220]:
a = description.find_all("p")

In [222]:
for b in a:
    print(b.text)

This Dynamic Web Design with PHP and MySQL training course will give you a working understanding of these important technologies and show you how to use them to create your own impressive websites. When used in combination, there simply isn't a better way to construct sites that involve complexity, large volumes of data, and the flexibility to make adjustments and changes over time. This course goes step by step through the basics of each technology, and then gradually shows you how to build the necessary components and underlying framework of your site. It goes through installation of PHP and MySQL technologies on your local machine as well as hosted servers, explains basic tags, commands and constructs, and helps you build and work within a solid back-end database. One you have the basics of PHP and MySQL covered, you'll also learn how to use CSS, JavaScript and AJAX to create the visual style and added functionality your pages deserve.
 
What You Will Learn 
- How to build functiona

In [210]:
for d in description:
    try:
        print(d.get('p'))
    except:
        pass

None


In [211]:
for i in range(0,len(description),2):
    print(description[i].get('p'))
    

None


### Ranking Distribution 

In [112]:
dist_rating = page_soup.find('div', class_ ='ud-component--clp--course-reviews-display')
dist_rating

<div class="ud-component--clp--course-reviews-display" data-component-args='{"ratingDistribution":[{"count":19,"rating":1},{"count":27,"rating":2},{"count":110,"rating":3},{"count":270,"rating":4},{"count":490,"rating":5}],"topReviewAttributes":[],"averageRating":4.5738,"showUserAvatar":true,"isFreeSEOExp":false,"courseId":28295,"page":"clp"}'>
</div>

In [114]:
dist_rating_str = dist_rating['data-component-args']
dist_rating_str

'{"ratingDistribution":[{"count":19,"rating":1},{"count":27,"rating":2},{"count":110,"rating":3},{"count":270,"rating":4},{"count":490,"rating":5}],"topReviewAttributes":[],"averageRating":4.5738,"showUserAvatar":true,"isFreeSEOExp":false,"courseId":28295,"page":"clp"}'

In [129]:
counts = re.findall(r'"count":\d+', dist_rating_str)
rating = re.findall(r'"rating":\d+', dist_rating_str)

In [139]:
count, rank = [], []
for c in counts:
    count.append(c[8:])
    
for rt in rating:
    rank.append(c[9:])
    
rating_dist = set(zip(count, rank))
print(rating_dist)

{('490', '90'), ('110', '90'), ('270', '90'), ('27', '90'), ('19', '90')}


### CourseID

In [120]:
courseId = re.findall(r'"courseId":\d+', dist_rating_str)
print(courseId)

['"courseId":28295']


### Average Rating

In [128]:
averageRating = re.findall(r'"averageRating":\d.\d+', dist_rating_str)
print(averageRating)

['"averageRating":4.5738']


### Requirements

In [126]:
requirements = page_soup.find_all('li', class_ ='requirements__item')
for requirement in requirements:
    print(requirement.text)

Adobe Photoshop CS3 or higher
Very basic knowledge of HTML & CSS
Text Editor (TextWranger, Espresso, or Coda recommended)


In [178]:
#page_soup.find_all('span')

In [177]:
#page_soup.find_all('li')

In [176]:
#page_soup.find_all('div', class_ ='clp-component-render')

## Automatization of web scraping

In [6]:
def reviewExtraction(soup):
    """
    Method description
    """
    try:
        review = soup.find('div', class_ ='ud-component--clp--featured-review-content')
        review = review.text
    except:
        review = None
    return review

In [7]:
def description(soup):
    """
    Method description
    """
    try:
        description = soup.find('div', class_ ='js-simple-collapse-inner')
        paragraphs = description.find_all("p")
                            
        sentences = []
        for sentence in paragraphs:
            sentences.append(sentence.text)
            
        description = ' '.join(sentences)
    except:
        description = None
    return 

In [8]:
def RatingExtraction(soup):
    """
    Method description
    """
    try:
        dist_rating = soup.find('div', class_ ='ud-component--clp--course-reviews-display')
        dist_rating_str = dist_rating['data-component-args']
        counts = re.findall(r'"count":\d+', dist_rating_str)
        rating = re.findall(r'"rating":\d+', dist_rating_str)
        count, rank = [], []
        for c in counts:
            count.append(c[8:])

        for rt in rating:
            rank.append(c[9:])

        rating_dist = set(zip(count, rank))
    except:
        rating_dist = None
    
    return rating_dist

In [9]:
def CourseId(soup):
    """
    Method description
    """
    try:
        dist_rating = soup.find('div', class_ ='ud-component--clp--course-reviews-display')
        dist_rating_str = dist_rating['data-component-args']
        courseId = re.findall(r'"courseId":\d+', dist_rating_str)
    except:
        courseId = None
    return courseId

In [10]:
def RequirementsExtraction(soup):
    """
    Method description
    """
    try:
        requirements = soup.find_all('li', class_ ='requirements__item')
        req = []
        for requirement in requirements:
            req.append(requirement.text)
    except:
        req = None
    return req

In [11]:
def webScraping(url):
    """
    Method description
    """
    req = Request(url, headers=headers) 
    webpage = urlopen(req).read()
    soup_page = soup(webpage, 'html.parser')
    
    description = description(soup_page)
    review = reviewExtraction(soup_page)
    rating = RatingExtraction(soup_page)
    courseId = CourseId(soup_page)
    requirements = RequirementsExtraction(soup_page)
    
    return description, review, rating, courseId, requirements

In [12]:
len_df = df_courses.shape[0]

In [32]:
df_chunk1 =  df_courses.loc[0:100, :]

In [33]:
df_chunk1.url

0      https://www.udemy.com/build-beautiful-html5-we...
1      https://www.udemy.com/learning-dynamic-website...
2                        https://www.udemy.com/chatbots/
3               https://www.udemy.com/projects-in-html5/
4                   https://www.udemy.com/html-css-more/
5      https://www.udemy.com/how-to-make-a-wordpress-...
6                   https://www.udemy.com/django-python/
7      https://www.udemy.com/angular-firebase-applica...
8      https://www.udemy.com/web-development-mastercl...
9      https://www.udemy.com/spring-boot-tutorial-for...
10                https://www.udemy.com/bootstrapcourse/
11          https://www.udemy.com/scaling-docker-on-aws/
12        https://www.udemy.com/learn-javascript-course/
13                    https://www.udemy.com/learn-html5/
14     https://www.udemy.com/learn-by-doing-vue-js-2-...
15     https://www.udemy.com/learn-complete-web-devel...
16         https://www.udemy.com/webdevelopment101_html/
17     https://www.udemy.com/we

In [ ]:
scraping_cols = []

for url in df_chunk1.url:
    scraping_cols.append(webScraping(url))
    time.sleep(15)

In [22]:
df_chunk1.url.apply(webScraping)

HTTPError: HTTP Error 403: Forbidden

In [ ]:
df.to_csv('..\Data\interim\df_chunk1.csv', index = False, header=True) 